<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [infinite monkeys typing for an infinite amount of time](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of Wiliam Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

This text file contains the complete works of Shakespeare: https://www.gutenberg.org/files/100/100-0.txt

Use it as training data for an RNN - you can keep it simple and train character level, and that is suggested as an initial approach.

Then, use that trained RNN to generate Shakespearean-ish text. Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) to generate, and returns generated text of that size.

Note - Shakespeare wrote an awful lot. It's OK, especially initially, to sample/use smaller data and parameters, so you can have a tighter feedback loop when you're trying to get things running. Then, once you've got a proof of concept - start pushing it more!

### Load Data

In [0]:
# TODO - Words, words, mere words, no matter from the heart.

In [0]:
import requests
import re
import string

In [0]:
shakespeare_url = 'https://www.gutenberg.org/cache/epub/100/pg100.txt'

# get the text
response = requests.get(shakespeare_url)
shakespeare_file = response.content

In [0]:
# decode binary into string
shakespeare_text = shakespeare_file.decode('utf-8')

In [132]:
# drop first few descriptive paragraphs.
# when dropping the first few descpritive parapgraphs we are still left with 
# over 5 million characters. Take a smaller sample of that... closer to the first
# 600k or so... 
shakespeare_text = shakespeare_text[7675:]
print(len(shakespeare_text))

5574537


In [0]:
punctuation = string.punctuation
punctuation = ''.join([x for x in punctuation if x not in ['-', "'", '.', "?", "!"]])

In [0]:
# remove newlines, whitespace, and some puncutation
shakespeare_text = shakespeare_text.replace('\r\n', '')
shakespeare_text = shakespeare_text.replace('\n', '')
shakespeare_text = re.sub(r'[{}]'.format (punctuation), ' ', shakespeare_text)
shakespeare_text = re.sub(' +', ' ', shakespeare_text).strip()

In [0]:
shakespeare_text = shakespeare_text[0:850000]

In [138]:
# The unique characters in the file
vocab = sorted(set(shakespeare_text))
print ('{} unique characters'.format(len(vocab)))

68 unique characters


### TODO

In [0]:
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import RMSprop

import tensorflow as tf
import pandas as pd
import numpy as np
import random
import sys
import os
import time

In [0]:
# Lookup Tables
char_int = {c:i for i, c in enumerate(vocab)} 
int_char = {i:c for i, c in enumerate(vocab)} 

In [143]:
# Create the sequence data

maxlen = 80
step = 1

encoded = [char_int[c] for c in shakespeare_text]

sequences = [] # Each element is 75 chars long
next_char = [] # One element for each sequence

for i in range(0, len(encoded) - maxlen, step):
    
    sequences.append(encoded[i : i + maxlen])
    next_char.append(encoded[i + maxlen])
    
print('sequences: ', len(sequences))

sequences:  849920


In [0]:
# Create x & y

x = np.zeros((len(sequences), maxlen, len(vocab)), dtype=np.bool)
y = np.zeros((len(sequences),len(vocab)), dtype=np.bool)

for i, sequence in enumerate(sequences):
    for t, char in enumerate(sequence):
        x[i,t,char] = 1
        
    y[i, next_char[i]] = 1

In [145]:
print(x.shape)
print(y.shape)

(849920, 80, 68)
(849920, 68)


### Build Model

In [0]:
# build the model: a single LSTM

model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(vocab)), activation='tanh', recurrent_activation='sigmoid', dropout=0.2, recurrent_dropout=0, unroll=False, use_bias=True))
model.add(Dense(len(vocab), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

In [147]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 128)               100864    
_________________________________________________________________
dense_2 (Dense)              (None, 68)                8772      
Total params: 109,636
Trainable params: 109,636
Non-trainable params: 0
_________________________________________________________________


In [0]:
def sample(preds):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / 1
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [0]:
def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    
    print()
    print('----- Generating text after Epoch: %d' % epoch)
    
    start_index = random.randint(0, len(shakespeare_text) - maxlen - 1)
    
    generated = ''
    
    sentence = shakespeare_text[start_index: start_index + maxlen]
    generated += sentence
    
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)
    
    for i in range(400):
        x_pred = np.zeros((1, maxlen, len(vocab)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_int[char]] = 1
            
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds)
        next_char = int_char[next_index]
        
        sentence = sentence[1:] + next_char
        
        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()


print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [150]:
# fit the model

model.fit(x, y,
          batch_size=512,
          epochs=10,
          callbacks=[print_callback])

Epoch 1/10
1659/1660 [============================>.] - ETA: 0s - loss: 2.5698
----- Generating text after Epoch: 0
----- Generating with seed: "om public haunt Finds tongues in trees books in the running brooks Sermons in st"
om public haunt Finds tongues in trees books in the running brooks Sermons in sthe geanrsorer INoll no rubl padorsy daiment rakaven laches. theve ofkberiks hon me tfoy a mo heaf y. fstoadn of frs oxt wile noe tisaps we hit sres the hor to fofe af ce firgot than me ars youllt comr woll pee mat. Et womes mat ro tie by hrus bave sirobn hem heareariig c. OF EONIhU. Taimertine. IORPAARI DI I Fir the coot ssur whim? Faat. Wrad bl! you wimg yonc unofgrorI ama kist f-whakt by rok tha
1660/1660 [==============================] - 86s 52ms/step - loss: 2.5697
Epoch 2/10
1660/1660 [==============================] - ETA: 0s - loss: 2.2657
----- Generating text after Epoch: 1
----- Generating with seed: "ill we be married. AUDREY. I do desire it with all my heart and I hope it

# Resources and Stretch Goals

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN